RAINSTORM

## Seize labels

- This notebook will help us visualize the results from labeled videos of an experiment

#### Requirements:

- The position files (processed by [2a-Prepare_positions](2a-Prepare_positions.ipynb))
- The movement files (processed by [2b-Geometric_analysis](2b-Geometric_analysis.ipynb))
- The labels, geolabels or autolabels files (labeled by [1-Behavioral_labeler](1-Behavioral_labeler.ipynb), [2b-Geometric_analysis](2b-Geometric_analysis.ipynb) or [3b-Automatic_analysis](3b-Automatic_analysis.ipynb))

---
#### Load the necessary modules

In [1]:
from pathlib import Path
import rainstorm.seize_labels as rst

---
#### 1. State your project path and experiment details in the params file
Define the path to the experiment folder, and the path to the parameters file.

Choose in `label_type` if you want to use the automatic labels or the geometric labels for target exploration analysis.

In [ ]:
base = Path.cwd()
folder_path = base / 'examples' / 'NOR'
params = folder_path / 'params.yaml'

label_type = 'autolabels'  # choose between 'autolabels' and 'geolabels' (or just 'labels' if you labeled manually)

---
#### 2. Visualize the behavioral labels on a video

The following function combines the position data, the behavioral labels, and the original video recording to create a video with the labels overlaid.

The video can be built both using the original video file or not... try removing the `video_path` argument from the function call to see the difference.

In [ ]:
example_path = rst.choose_example_positions(params, look_for = 'TS_03', suffix='_positions.csv')

In [ ]:
video_name = example_path.name.replace('_positions.csv', '.mp4')
video_path = folder_path / 'videos' / video_name
# rst.create_video(params, example_path, video_path, label_type=label_type)

---
#### 3. Plot individual exploration


Exploration is a dynamic behavior, and experiment results can change a lot depending on the timeframe chosen for analysis. A first step to understanding the exploration dynamics is to plot the evolution of the cumulative exploration time.

In [ ]:
rst.plot_mouse_exploration(params, example_path, label_type=label_type)

This plot shows the behavior of a single subject, and does not take into account the role of each exploratory target. To build more elaborate plots, we need to inform the groups and target roles for each mice and trial.

---
#### 4. Create a ‘reference’ file on our experiment folder.
This function will create a ‘reference.json’ file that will be used to organize the files and plot the experiment results.

In [ ]:
reference_path = rst.create_reference_file(params, overwrite=False)

#### Use the Reference Editor GUI
Instead of manually editing the file, you can now use the new Reference Editor GUI to:
- Define groups for your experiment  
- Configure target roles for each trial
- Assign groups, target roles, and ROI roles to each video file

In [ ]:
rst.open_reference_editor(reference_path)

If you are using the NOR example folder to try out rainstorm, you will find that **the reference file is already complete** (no need to change anything).

Once the reference.json file is complete, you can proceed to the next step where it will be used to create the 'summary' folder.

---
#### 5. Create the 'Summary' folder
This new folder will contain files for each video recording, which will be used to hold all the labels and geometric information.

The groups and objects on the ‘reference.json’ file will be used to organize the files into subfolders, and the target columns will be renamed according to their role (e.g. Novel and Known).

In [ ]:
summary_path = rst.create_summary_files(params, label_type=label_type, overwrite=True)

---
#### 6. With our files ready, all is left to do is to run the analysis and plot the results.

We can plot...

For all trials:
- lineplot_cumulative_distance
- lineplot_cumulative_freezing_time

For trials with targets:
- lineplot_cumulative_exploration_time
- boxplot_total_exploration_time

Some calculated index:
- lineplot_DI
- lineplot_diff
- boxplot_DI_auc (Area under the curve of the lineplot_DI)
- boxplot_avg_time_bias (An "average rate of bias" obtained by dividing the diff AUC by the total session duration)

And finally, some ROIs realted plots:
- boxplot_roi_time
- boxplot_roi_distance
- boxplot_alternation_proportion

In [ ]:
outliers = [] # List of video names to exclude from analysis (e.g., 'TS_09', or just '09' to remove that subject from all trials)

rst.plot_multiple_analyses(params, trial='Hab', plots=[rst.lineplot_cumulative_distance, rst.lineplot_cumulative_freezing_time], label_type=label_type, outliers=outliers)
rst.plot_multiple_analyses(params, trial='TR', plots=[rst.boxplot_total_exploration_time, rst.boxplot_roi_distance], label_type=label_type, outliers=outliers)
rst.plot_multiple_analyses(params, trial='TS', plots=[rst.lineplot_cumulative_exploration_time, rst.lineplot_DI, rst.boxplot_avg_time_bias], label_type=label_type, outliers=outliers)

#### These modular graphs are intended to show the dynamic behavior of mice, providing a deeper insight into what happens during learning and memory.


---
#### 7. Make a csv file with the results of the experiment

The results.csv file will be stored in the experiment folder, and will contain columns for each of the variables plotted above. It is meant to be used for further statistical analysis.

In [ ]:
rst.create_results_file(params, label_type=label_type, overwrite=True)

---
#### 8. Bonus track... For each mouse, create individual plots

This function is thought particularly for experiments like Novel Object Recognition, since it works better if we have two targets on the arena.
The individual plots give us the chance to evaluate if something is off, and spot any behavioral outliers on both training and testing sessions.

You can test it with the demo data without problems, the plots will be stored in the example folder.

In [ ]:
rst.run_individual_analysis(params, label_type=label_type, show=False)


We have reached the end of this project. If you have read and followed the code up to this point, I want to thank you for your time and attention.

If you have any questions and/or suggestions, do not hesitate to write: sdhers@fbmc.fcen.uba.ar


---
RAINSTORM - Created on Jun 25, 2024 - @author: Santiago D'hers